<a href="https://colab.research.google.com/github/aasthadawar02/Sample/blob/main/Kaggle_Dataset_Mobile_V2_vs_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import PIL.Image as Image
import tensorflow_datasets as tfds
import pandas as pd

In [2]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.metrics import accuracy_score

In [3]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
train_directory="/content/drive/MyDrive/TME_6015/Assignment_2/Big_Cats_Kaggle/train"

train_data = tf.keras.utils.image_dataset_from_directory(train_directory,
                                                         validation_split=None,
                                                         image_size=(128, 128),
                                                         batch_size=16)

Found 2339 files belonging to 10 classes.


In [5]:
train_data

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
validation_directory="/content/drive/MyDrive/TME_6015/Assignment_2/Big_Cats_Kaggle/valid"
validation_data = tf.keras.utils.image_dataset_from_directory(validation_directory,
                                                         validation_split=None,
                                                         image_size=(128, 128),
                                                         batch_size=16)

Found 50 files belonging to 10 classes.


In [8]:
validation_data

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [10]:
test_directory="/content/drive/MyDrive/TME_6015/Assignment_2/Big_Cats_Kaggle/test"
test_data = tf.keras.utils.image_dataset_from_directory(test_directory,
                                                         validation_split=None,
                                                         image_size=(128, 128),
                                                         batch_size=16)

Found 50 files belonging to 10 classes.


In [11]:
test_data

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [12]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [18]:
model_handle = "https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/035-128-classification/versions/2"

feature_extractor = hub.KerasLayer(model_handle,
                                   trainable=True)
num_class = 10
model = tf.keras.Sequential(
    [
        normalization_layer,
        feature_extractor,
        tf.keras.layers.Dense(num_class, activation="softmax")
    ]
)
model.build([None, 128, 128, 3])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 keras_layer_1 (KerasLayer)  (None, 1001)              1692489   
                                                                 
 dense_1 (Dense)             (None, 10)                10020     
                                                                 
Total params: 1702509 (6.49 MB)
Trainable params: 1688429 (6.44 MB)
Non-trainable params: 14080 (55.00 KB)
_________________________________________________________________


In [20]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
EPOCHS = 25
history = model.fit(train_data,
                    epochs=EPOCHS,
                    validation_data=validation_data)

Epoch 1/25
147/147 [==============================] - 271s 2s/step - loss: 2.1045 - accuracy: 0.5981 - val_loss: 2.2919 - val_accuracy: 0.5400
Epoch 2/25
147/147 [==============================] - 34s 228ms/step - loss: 0.7447 - accuracy: 0.7918 - val_loss: 1.7093 - val_accuracy: 0.6800
Epoch 3/25
147/147 [==============================] - 33s 225ms/step - loss: 0.5794 - accuracy: 0.8444 - val_loss: 1.4971 - val_accuracy: 0.6800
Epoch 4/25
147/147 [==============================] - 34s 233ms/step - loss: 0.6300 - accuracy: 0.8307 - val_loss: 0.8630 - val_accuracy: 0.7800
Epoch 5/25
147/147 [==============================] - 35s 234ms/step - loss: 0.5782 - accuracy: 0.8401 - val_loss: 1.3784 - val_accuracy: 0.7000
Epoch 6/25
147/147 [==============================] - 34s 233ms/step - loss: 0.4671 - accuracy: 0.8782 - val_loss: 0.6221 - val_accuracy: 0.8400
Epoch 7/25
147/147 [==============================] - 33s 224ms/step - loss: 0.4875 - accuracy: 0.8794 - val_loss: 0.6862 - val_accu

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(validation_data)

In [ ]:
print(test_acc)

In [ ]:
images=[]
labels=[]
for image, label in test_data:
  images.extend(image.numpy())
  labels.extend(label.numpy())
predicted_labels = model.predict(np.array(images))
predicted_labels = list(map(lambda val : np.argmax(val), predicted_labels))
len(predicted_labels)

In [ ]:
len(predicted_labels)

In [ ]:
num_classes =train_data.class_names

In [ ]:
plt.figure(figsize=(10,10))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.subplots_adjust(hspace = 0.3)
    plt.imshow(images[i].astype("uint8"))
    color = "blue" if predicted_labels[i] == labels[i] else "red"
    plt.title(num_classes[predicted_labels[i]].title(), color=color)
    plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,ConfusionMatrixDisplay,classification_report

In [ ]:
print(classification_report(labels,predicted_labels))

In [ ]:
ConfusionMatrixDisplay.from_predictions(labels, predicted_labels, display_labels=num_classes, xticks_rotation="vertical")
plt.show()

In [ ]:
train_directory="/content/drive/MyDrive/TME_6015/Assignment_2/Big_Cats_Kaggle/train"

train_data = tf.keras.utils.image_dataset_from_directory(train_directory,
                                                         validation_split=None,
                                                         image_size=(128, 128),
                                                         batch_size=16)

In [ ]:
train_data

In [ ]:
validation_directory="/content/drive/MyDrive/TME_6015/Assignment_2/Big_Cats_Kaggle/valid"
validation_data = tf.keras.utils.image_dataset_from_directory(validation_directory,
                                                         validation_split=None,
                                                         image_size=(128, 128),
                                                         batch_size=16)

In [ ]:
validation_data

In [ ]:
test_directory="/content/drive/MyDrive/TME_6015/Assignment_2/Big_Cats_Kaggle/test"
test_data = tf.keras.utils.image_dataset_from_directory(test_directory,
                                                         validation_split=None,
                                                         image_size=(128, 128),
                                                         batch_size=16)

In [ ]:
test_data

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
model_handle = "https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/035-128-classification/versions/2"

feature_extractor = hub.KerasLayer(model_handle,
                                   trainable=True)
num_class = 10
model = tf.keras.Sequential(
    [
        normalization_layer,
        feature_extractor,
        tf.keras.layers.Dense(num_class, activation="softmax")
    ]
)
model.build([None, 128, 128, 3])

In [ ]:
model.summary()

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
EPOCHS = 25
history = model.fit(train_data,
                    epochs=EPOCHS,
                    validation_data=validation_data)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(validation_data)

In [ ]:
print(test_acc)

In [ ]:
images=[]
labels=[]
for image, label in test_data:
  images.extend(image.numpy())
  labels.extend(label.numpy())
predicted_labels = model.predict(np.array(images))
predicted_labels = list(map(lambda val : np.argmax(val), predicted_labels))
len(predicted_labels)

In [ ]:
len(predicted_labels)

In [ ]:
num_classes =train_data.class_names

In [ ]:
plt.figure(figsize=(10,10))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.subplots_adjust(hspace = 0.3)
    plt.imshow(images[i].astype("uint8"))
    color = "blue" if predicted_labels[i] == labels[i] else "red"
    plt.title(num_classes[predicted_labels[i]].title(), color=color)
    plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,ConfusionMatrixDisplay,classification_report

In [ ]:
print(classification_report(labels,predicted_labels))

In [ ]:
ConfusionMatrixDisplay.from_predictions(labels, predicted_labels, display_labels=num_classes, xticks_rotation="vertical")
plt.show()